<a href="https://colab.research.google.com/github/ogigo/image_segmentation/blob/main/Pneumothoeax_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
!mkdir -p ~/.kaggle

In [3]:
api_token = {"username":"kajolmaster","key":"dbb23ceaca2a39397c3e92d83fda1034"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:json.dump(api_token, file)

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download jesperdramsch/siim-acr-pneumothorax-segmentation-data

100% 3.03G/3.03G [01:36<00:00, 40.4MB/s]
100% 3.03G/3.03G [01:37<00:00, 33.6MB/s]


In [6]:
!kaggle datasets download abhishek/siim-png-images

100% 9.00G/9.00G [04:47<00:00, 39.0MB/s]
100% 9.00G/9.00G [04:48<00:00, 33.6MB/s]


In [ ]:
!unzip siim-acr-pneumothorax-segmentation-data.zip

In [ ]:
!unzip siim-png-images.zip

In [ ]:
!pip install segmentation-models-pytorch

In [35]:
import os
import random
import glob
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict, Counter

from PIL import Image,ImageFile
import albumentations as A
import matplotlib.pyplot as plt

from sklearn import model_selection
import segmentation_models_pytorch as smp

import torch
from torch import nn,optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

ImageFile.LOAD_TRUNCATED_IMAGES = True
# device="cpu"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
def set_seed(seed = 42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(101)

> SEEDING DONE


In [11]:
train_data=pd.read_csv("/content/pneumothorax/train-rle.csv")

In [12]:
train_data.head()

,ImageId,EncodedPixels
0,1.2.276.0.7230010.3.1.4.8323329.5597.151787518...,-1
1,1.2.276.0.7230010.3.1.4.8323329.12515.15178752...,-1
2,1.2.276.0.7230010.3.1.4.8323329.4904.151787518...,175349 7 1013 12 1009 17 1005 19 1003 20 1002...
3,1.2.276.0.7230010.3.1.4.8323329.32579.15178751...,407576 2 1021 7 1015 10 1013 12 1011 14 1008 ...
4,1.2.276.0.7230010.3.1.4.8323329.32579.15178751...,252069 1 1021 3 1020 4 1018 5 1018 6 1016 7 1...


In [37]:
train_df,val_df = model_selection.train_test_split(train_data, test_size=0.15)

In [21]:
train_data.ImageId.values

array(['1.2.276.0.7230010.3.1.4.8323329.5597.1517875188.959090',
       '1.2.276.0.7230010.3.1.4.8323329.12515.1517875239.501137',
       '1.2.276.0.7230010.3.1.4.8323329.4904.1517875185.355709', ...,
       '1.2.276.0.7230010.3.1.4.8323329.32730.1517875162.25023',
       '1.2.276.0.7230010.3.1.4.8323329.13252.1517875244.359912',
       '1.2.276.0.7230010.3.1.4.8323329.12050.1517875237.113402'],
      dtype=object)

In [22]:
sample=train_data.groupby("ImageId")

In [23]:
sample.first().head()

,EncodedPixels
ImageId,
1.2.276.0.7230010.3.1.4.8323329.1000.1517875165.878027,-1
1.2.276.0.7230010.3.1.4.8323329.10000.1517875220.938530,-1
1.2.276.0.7230010.3.1.4.8323329.10001.1517875220.930580,-1
1.2.276.0.7230010.3.1.4.8323329.10002.1517875220.939397,-1
1.2.276.0.7230010.3.1.4.8323329.10003.1517875220.942420,-1


In [24]:
sample_2=sample.get_group(train_data["ImageId"][1])

In [25]:
sample_2

,ImageId,EncodedPixels
1,1.2.276.0.7230010.3.1.4.8323329.12515.15178752...,-1


In [26]:
sample_2[" EncodedPixels"]

1     -1
Name:  EncodedPixels, dtype: object

In [27]:
def run_length_decode(rle, height=1024, width=1024, fill_value=1):
    component = np.zeros((height, width), np.float32)
    component = component.reshape(-1)
    rle = np.array([int(s) for s in rle.strip().split(' ')])
    rle = rle.reshape(-1, 2)
    start = 0
    for index, length in rle:
        start = start+index
        end = start+length
        component[start: end] = fill_value
        start = end
    component = component.reshape(width, height).T
    return component

In [50]:
class SIIMDataset(Dataset):
    
    def __init__(self, df, data_dir, transform=None, preprocessing_fun=None, channel_first=True):
        self.data_dir = data_dir
        self.transform = transform                       # for augmentations
        self.preprocessing_fun = preprocessing_fun       # preprocessing_fun to normalize images
        self.channel_first = channel_first               # set channels as first dimension
        self.image_ids = df.ImageId.values
        self.group_by = df.groupby('ImageId')

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        df = self.group_by.get_group(img_id)
        annotations = df[' EncodedPixels'].tolist()
        
        img_path = os.path.join(self.data_dir, img_id + ".png")
        img = Image.open(img_path).convert('RGB')
        img = np.array(img)

        mask = np.zeros(shape=(1024,1024))
        if annotations[0] != ' -1':
            for rle in annotations:
                mask += run_length_decode(rle)
        mask = (mask >= 1).astype('float32')
        mask = np.expand_dims(mask, axis=-1)
        
        # apply augmentation
        if self.transform:
            augmented = self.transform(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']

        if self.preprocessing_fun:
            img = self.preprocessing_fun(img)
        
        # convert shape from (width, height, channel) ----> (channel, width, height) 
        if self.channel_first:
            img = np.transpose(img, (2, 0, 1)).astype(np.float32)
            mask = np.transpose(mask, (2, 0, 1)).astype(np.float32)

        return {
            'image': torch.Tensor(img),
            'mask': torch.Tensor(mask)
        }

In [29]:
train_image_dir="/content/train_png"
batch_size=32

encoder="resnet34"
weights="imagenet"

In [31]:
transform = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.OneOf([A.RandomGamma(gamma_limit=(90,110)),
             A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)], p=0.5),
    A.Resize(width=224, height=224)
])

In [34]:
prep_fun = smp.encoders.get_preprocessing_fn(encoder,weights)

In [51]:
train_dataset=SIIMDataset(train_df,
                          train_image_dir,
                          transform=transform,
                          preprocessing_fun=prep_fun)

val_dataset=SIIMDataset(val_df,
                        train_image_dir,
                        preprocessing_fun=prep_fun,
                        transform=transform)

In [52]:
train_loader = DataLoader(train_dataset,
                          batch_size = batch_size,
                          shuffle = True,
                          num_workers = 0)

val_loader = DataLoader(val_dataset,
                        batch_size = batch_size,
                        num_workers = 0)

In [53]:
print('Training data Info:')
dataiter = iter(train_loader)
data = next(dataiter)
images,labels = data['image'],data['mask']
print("shape of images : {}".format(images.shape))
print("shape of labels : {}".format(labels.shape))

Training data Info:
shape of images : torch.Size([32, 3, 224, 224])
shape of labels : torch.Size([32, 1, 224, 224])


In [62]:
def dice_loss(input, target):
    input = torch.sigmoid(input)
    smooth = 1.0
    iflat = input.view(-1)
    tflat = target.view(-1)
    intersection = (iflat * tflat).sum()
    return ((2.0 * intersection + smooth) / (iflat.sum() + tflat.sum() + smooth))

In [63]:
class FocalLoss(nn.Module):
    def __init__(self, gamma):
        super().__init__()
        self.gamma = gamma

    def forward(self, input, target):
        if not (target.size() == input.size()):
            raise ValueError("Target size ({}) must be the same as input size ({})"
                             .format(target.size(), input.size()))
        max_val = (-input).clamp(min=0)
        loss = input - input * target + max_val + \
            ((-max_val).exp() + (-input - max_val).exp()).log()
        invprobs = F.logsigmoid(-input * (target * 2.0 - 1.0))
        loss = (invprobs * self.gamma).exp() * loss
        return loss.mean()

In [64]:
class MixedLoss(nn.Module):
    def __init__(self, alpha, gamma):
        super().__init__()
        self.alpha = alpha
        self.focal = FocalLoss(gamma)

    def forward(self, input, target):
        loss = self.alpha*self.focal(input, target) - torch.log(dice_loss(input, target))
        return loss.mean()

In [65]:
model = smp.Unet(
    encoder_name = encoder,
    encoder_weights = weights,
    in_channels = 3,
    classes = 1,
    activation = None
)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [66]:
model.to(device)

Unet(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [68]:
def train(data_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0
    for data in tqdm(data_loader):
        inputs = data['image']
        labels = data['mask']

        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    return train_loss/len(data_loader)

In [69]:
def evaluate(data_loader, model, criterion):
    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for data in tqdm(data_loader):
            inputs = data['image']
            labels = data['mask']

            inputs = inputs.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            eval_loss += loss.item()

    return eval_loss/len(data_loader)

In [67]:
criterion = MixedLoss(alpha = 10.0,
                      gamma = 2.0)

optimizer = optim.Adam(model.parameters(),
                       lr = 0.0001)

In [70]:
epochs = 15

for epoch in range(epochs):

    train_loss = train(train_loader,
                       model,
                       criterion,
                       optimizer)

    val_loss = evaluate(val_loader,
                        model,
                        criterion)

    print(f'Epoch: {epoch+1}')
    print(f'Training Loss: {train_loss}, \t Validation Loss: {val_loss}\n')


100%|██████████| 55/55 [00:52<00:00,  1.05it/s]


Epoch: 1
Training Loss: 3.8050252725551656, 	 Validation Loss: 3.0508282618089155



100%|██████████| 55/55 [00:50<00:00,  1.09it/s]


Epoch: 2
Training Loss: 2.000528137211676, 	 Validation Loss: 1.996427221731706



100%|██████████| 55/55 [00:49<00:00,  1.12it/s]


...

Epoch: 13
Training Loss: 0.4912796408518568, 	 Validation Loss: 1.3125843720002608



100%|██████████| 55/55 [00:49<00:00,  1.10it/s]


Epoch: 14
Training Loss: 0.6158059009096839, 	 Validation Loss: 1.5146106492389333



100%|██████████| 55/55 [00:49<00:00,  1.11it/s]

Epoch: 15
Training Loss: 0.6670085581672656, 	 Validation Loss: 1.3106813398274508

